In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from sklearn.metrics import roc_curve
import numpy as np
import timm

In [2]:
# Define the transformations for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Define paths to your data folders
# Define paths to your data folders
train_data_dir = '/u/45/muhammu2/data/Desktop/sample/train/'
val_data_dir = '/u/45/muhammu2/data/Desktop/sample/validation/'
test_data_dir = '/u/45/muhammu2/data/Desktop/sample/test/'

# Load your training, validation, and test datasets using ImageFolder
train_dataset = ImageFolder(root=train_data_dir, transform=transform)
val_dataset = ImageFolder(root=val_data_dir, transform=transform)
test_dataset = ImageFolder(root=test_data_dir, transform=transform)

# Create data loaders
batch_size = 16  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# Define your SimpleDenseNet model (as defined previously)
class SimpleDenseNet(nn.Module):
    def __init__(self, num_classes):
        super(SimpleDenseNet, self).__init__()
        self.model = timm.create_model('densenet201', pretrained=True)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Instantiate the custom DenseNet model
num_classes = 2  # Binary classification
model = SimpleDenseNet(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [4]:
# Train the model
num_epochs = 5 # Adjust the number of epochs as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

print("Finished Training")

Epoch 1, Loss: 0.48235172894265915
Epoch 2, Loss: 0.27324471059772704
Epoch 3, Loss: 0.13243597563770082
Epoch 4, Loss: 0.06818672004673217
Epoch 5, Loss: 0.030101422406733035
Finished Training


In [5]:
# Evaluate the model and calculate EER on the validation set
model.eval()
val_labels, val_scores = [], []

with torch.no_grad():
    for data, labels in val_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = model(data)
        val_scores.extend(outputs[:, 1].cpu().numpy())  # Assuming you're interested in class 1 scores
        val_labels.extend(labels.cpu().numpy())

# Calculate Equal Error Rate (EER) on the validation set
fpr, tpr, thresholds = roc_curve(val_labels, val_scores, pos_label=1)
eer_threshold = thresholds[np.argmin(np.abs(tpr - (1 - fpr)))]
eer = 1 - tpr[np.argmin(np.abs(tpr - (1 - fpr)))]
print(f'Validation EER: {eer*100:.4f}, EER Threshold: {eer_threshold:.4f}')

Validation EER: 1.9048, EER Threshold: -0.5025


In [6]:
# Calculate HTER on the test set using the EER threshold
test_labels, test_scores = [], []

with torch.no_grad():  # Ensure gradients are not tracked
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = model(data)
        test_scores.extend(outputs[:, 1].detach().cpu().numpy())  # Detach and convert to NumPy
        test_labels.extend(labels.cpu().numpy())

# Calculate the HTER on the testing set using the EER threshold
threshold = eer_threshold
predicted_labels = [1 if score > threshold else 0 for score in test_scores]

false_acceptance = sum(1 for i in range(len(predicted_labels)) if predicted_labels[i] == 1 and test_labels[i] == 0)
false_rejection = sum(1 for i in range(len(predicted_labels)) if predicted_labels[i] == 0 and test_labels[i] == 1)

total_samples = len(test_labels)
hter = ((false_acceptance + false_rejection) / (2 * total_samples)) * 100
print(f"HTER: {hter:.2f}%")




HTER: 26.25%


In [8]:
# Calculate the AUC on the testing set
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(test_labels, test_scores)
print(f"AUC on the testing set: {auc_score*100:.2f}")

AUC on the testing set: 81.62
